# Modeling

In [16]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re
from scipy import stats

In [17]:
df = pd.read_csv('../data/PostBooksEDA.csv', index_col=0)
df_encoded = pd.read_csv('../data/PostEncodedBooksEDA.csv', index_col=0)

In [18]:
df_encoded = df_encoded.drop(columns=['user_id','isbn'], axis=1)

For the initial models we will be building, User ID and ISBN will not be needed since they are merely identifiers and do not have meaningful relationships with the target variable.

## Stats Logistic model 

Since logistic models predict values between 0 and 1, we will convert the ratings into implicit (0) and explicit (1) categories to predict whether users are likely to leave a review or not.

In [21]:
X = df_encoded.drop('rating', axis=1)
y = df_encoded['rating'].copy()

In [19]:
X.columns

Index(['user_id', 'age', 'isbn', 'year_of_publication', 'Africa', 'Asia',
       'Europe', 'North America', 'Oceania', 'South America',
       'author_frequency', 'author_implicit_encoded',
       'author_explicit_encoded', 'author_avg_all_reviews',
       'publisher_frequency', 'publisher_implicit_encoded',
       'publisher_explicit_encoded', 'publisher_avg_all_reviews', 'de', 'en',
       'es', 'fr', 'it', 'unknown_lg', 'art & culture', 'business & economics',
       'children's books', 'cookbooks', 'fiction', 'health & fitness',
       'non-fiction', 'poetry', 'religion & spirituality',
       'science & technology', 'self-help', 'unknown_category'],
      dtype='object')

In [22]:
y = y.apply(lambda x: 1 if 1 <= x <= 10 else x)

In [23]:
y

0          0
1          1
2          0
4          0
5          1
          ..
1031170    0
1031171    1
1031172    1
1031173    1
1031174    1
Name: rating, Length: 978859, dtype: int64

In [24]:
import statsmodels.api as sm 

# 0. ADD THAT CONSTANT 
X_const = sm.add_constant(X)

# 1. Instatiate the model
bank_logit = sm.Logit(y, X_const)

# 2. Fit the model to the data
bank_logit_fitted = bank_logit.fit()

# 3. Look at results (Summary table)
bank_logit_fitted.summary()

Optimization terminated successfully.
         Current function value: 0.573486
         Iterations 6


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                 rating   No. Observations:               978859
Model:                          Logit   Df Residuals:                   978827
Method:                           MLE   Df Model:                           31
Date:                Thu, 21 Nov 2024   Pseudo R-squ.:                  0.1265
Time:                        23:23:01   Log-Likelihood:            -5.6136e+05
converged:                       True   LL-Null:                   -6.4268e+05
Covariance Type:            nonrobust   LLR p-value:                     0.000
==============================================================================================
                                 coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------------------
const                         -3.7907        nan        nan        nan         nan         nan
age                           -0.0011      0.000     -4.825      0.000      -0.002      -0.001
year_of_publication            0.0049      0.000     14.907      0.000       0.004       0.006
Africa                        -0.1527        nan        nan        nan         nan         nan
Asia                          -0.6341        nan        nan        nan         nan         nan
Europe                        -0.8208        nan        nan        nan         nan         nan
North America                 -0.8469        nan        nan        nan         nan         nan
Oceania                       -0.9085        nan        nan        nan         nan         nan
South America                 -0.4277        nan        nan        nan         nan         nan
author_frequency            5.562e-06   1.49e-06      3.722      0.000    2.63e-06    8.49e-06
author_implicit_encoded       -5.0709      0.120    -42.285      0.000      -5.306      -4.836
author_explicit_encoded       -0.0504      0.007     -7.165      0.000      -0.064      -0.037
author_avg_all_reviews         0.0323      0.016      2.061      0.039       0.002       0.063
publisher_frequency          3.54e-06   2.51e-07     14.084      0.000    3.05e-06    4.03e-06
publisher_implicit_encoded    -1.2668      0.250     -5.070      0.000      -1.756      -0.777
publisher_explicit_encoded    -0.1058      0.014     -7.325      0.000      -0.134      -0.077
publisher_avg_all_reviews      0.0667      0.033      2.035      0.042       0.002       0.131
de                            -0.6272        nan        nan        nan         nan         nan
en                            -0.5497        nan        nan        nan         nan         nan
es                            -0.5842        nan        nan        nan         nan         nan
fr                            -0.6744        nan        nan        nan         nan         nan
it                            -0.8564        nan        nan        nan         nan         nan
unknown_lg                    -0.4988        nan        nan        nan         nan         nan
art & culture                 -0.2947        nan        nan        nan         nan         nan
business & economics          -0.3799        nan        nan        nan         nan         nan
children's books              -0.2753        nan        nan        nan         nan         nan
cookbooks                     -0.3033        nan        nan        nan         nan         nan
fiction                       -0.2540        nan        nan        nan         nan         nan
health & fitness              -0.2377        nan        nan        nan         nan         nan
non-fiction                   -0.3402        nan        nan        nan         nan         nan
poetry                        -0.2921        nan        nan        nan         nan         nan
religion & 

In [25]:
# Calculate soft predictions
y_proba = bank_logit_fitted.predict(X_const)

# Convert soft predictions to hard predictions 0/1 
y_pred = np.where(y_proba >= 0.5, 1, 0)

# Calculate # correct
num_correct = (y_pred == y).sum()

# Calculate the percentage accuracy
pct_accuracy = num_correct/X.shape[0]

print(f'The baseline model accuracy is {np.round(pct_accuracy*100.0, 3)}%')

The baseline model accuracy is 68.326%
